# Raw Data Compiler

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display full data
pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment

'warn'

## Import and Compile Election Dataset

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/42MVDX

In [2]:
df = pd.read_csv('data/raw_data/election_data_2000-2016.csv')
df = df[(df['party'] == 'democrat') | (df['party'] == 'republican')]
df['percentage'] = df['candidatevotes']/df['totalvotes']
df = df[df['state'] != 'Hawaii']
df = df[df['state'] != 'Alaska']
df = df[df['county'] != 'Statewide writein']
df = df[df['county'] != 'Federal Precinct']
df = df[df['county'] != 'Maine UOCAVA']
df = df[df['FIPS'] != 36000] # remove Missouri Kansas City

In [3]:
# Add a new column

winner=[]
for i in range(len(df['state'])):
    winner.append('')
df['winner'] = winner

In [4]:
df.tail()

year    state state_po    county     FIPS     office        candidate  \
49851  2016  Wyoming       WY     Uinta  56041.0  President     Donald Trump   
49853  2016  Wyoming       WY  Washakie  56043.0  President  Hillary Clinton   
49854  2016  Wyoming       WY  Washakie  56043.0  President     Donald Trump   
49856  2016  Wyoming       WY    Weston  56045.0  President  Hillary Clinton   
49857  2016  Wyoming       WY    Weston  56045.0  President     Donald Trump   

            party  candidatevotes  totalvotes   version  percentage winner  
49851  republican          6154.0        8470  20191203    0.726564         
49853    democrat           532.0        3814  20191203    0.139486         
49854  republican          2911.0        3814  20191203    0.763241         
49856    democrat           299.0        3526  20191203    0.084799         
49857  republican          3033.0        3526  20191203    0.860182

In [5]:
# Add data on which party won the county

i = 0
for a, b in enumerate(df['FIPS']):
    try:
        if df.iloc[i, 8] > df.iloc[i+1, 8]:
            df.iloc[i, 12] = 'W'
            df.iloc[i+1, 12] = 'L'
        elif df.iloc[i, 8] < df.iloc[i+1, 8]:
            df.iloc[i, 12] = 'L'
            df.iloc[i+1, 12] = 'W'
    except:
        break
    i += 2

In [6]:
df.tail()

year    state state_po    county     FIPS     office        candidate  \
49851  2016  Wyoming       WY     Uinta  56041.0  President     Donald Trump   
49853  2016  Wyoming       WY  Washakie  56043.0  President  Hillary Clinton   
49854  2016  Wyoming       WY  Washakie  56043.0  President     Donald Trump   
49856  2016  Wyoming       WY    Weston  56045.0  President  Hillary Clinton   
49857  2016  Wyoming       WY    Weston  56045.0  President     Donald Trump   

            party  candidatevotes  totalvotes   version  percentage winner  
49851  republican          6154.0        8470  20191203    0.726564      W  
49853    democrat           532.0        3814  20191203    0.139486      L  
49854  republican          2911.0        3814  20191203    0.763241      W  
49856    democrat           299.0        3526  20191203    0.084799      L  
49857  republican          3033.0        3526  20191203    0.860182      W

In [7]:
# Sort for winning counties

df = df[df['winner'] == 'W']

In [8]:
# Replace W's with the winning party

for i, v in enumerate(df['party']):
    if v == 'republican':
        df.iloc[i, 12] = 'R'
    elif v == 'democrat':
        df.iloc[i, 12] = 'D'

In [9]:
# Add a column with the winning party of the corresponding election

election_winner=[]
for i, v in enumerate(df['year']):
    if v == 2000:
        election_winner.append('R')
    elif v == 2004:
        election_winner.append('R')
    elif v == 2008:
        election_winner.append('D')
    elif v == 2012:
        election_winner.append('D')
    elif v == 2016:
        election_winner.append('R')
df['election_winner'] = election_winner

In [10]:
df.head()

year    state state_po   county    FIPS     office       candidate  \
1   2000  Alabama       AL  Autauga  1001.0  President  George W. Bush   
5   2000  Alabama       AL  Baldwin  1003.0  President  George W. Bush   
8   2000  Alabama       AL  Barbour  1005.0  President         Al Gore   
13  2000  Alabama       AL     Bibb  1007.0  President  George W. Bush   
17  2000  Alabama       AL   Blount  1009.0  President  George W. Bush   

         party  candidatevotes  totalvotes   version  percentage winner  \
1   republican         11993.0       17208  20191203    0.696943      R   
5   republican         40872.0       56480  20191203    0.723654      R   
8     democrat          5188.0       10395  20191203    0.499086      D   
13  republican          4273.0        7101  20191203    0.601746      R   
17  republican         12667.0       17973  20191203    0.704779      R   

   election_winner  
1                R  
5                R  
8                R  
13               R  
17               R

In [11]:
# Count number of correct predictions for each county

county_wins = {}
for i, v in enumerate(df['FIPS']):
    if list(df['winner'])[i] == list(df['election_winner'])[i]:
        if int(v) not in county_wins.keys():
            county_wins[int(v)] = 1
        elif int(v) in county_wins.keys():
            county_wins[int(v)] += 1

In [12]:
county_wins

{1001: 3,
 1003: 3,
 1007: 3,
 1009: 3,
 1013: 3,
 1015: 3,
 1017: 3,
 1019: 3,
 1021: 3,
 1025: 3,
 1027: 3,
 1029: 3,
 1031: 3,
 1037: 3,
 1039: 3,
 1041: 3,
 1043: 3,
 1045: 3,
 1049: 3,
 1051: 3,
 1053: 3,
 1055: 3,
 1057: 3,
 1059: 3,
 1061: 3,
 1067: 3,
 1069: 3,
 1073: 4,
 1075: 3,
 1077: 3,
 1081: 3,
 1083: 3,
 1089: 3,
 1093: 3,
 1095: 3,
 1097: 3,
 1099: 3,
 1103: 3,
 1107: 3,
 1109: 3,
 1111: 3,
 1115: 3,
 1117: 3,
 1121: 3,
 1123: 3,
 1125: 3,
 1127: 3,
 1129: 3,
 1133: 3,
 4003: 3,
 4007: 3,
 4009: 3,
 4011: 3,
 4012: 3,
 4013: 3,
 4015: 3,
 4017: 3,
 4021: 3,
 4025: 3,
 4027: 3,
 5001: 3,
 5005: 3,
 5007: 3,
 5009: 3,
 5013: 3,
 5015: 3,
 5023: 3,
 5025: 3,
 5027: 3,
 5029: 3,
 5033: 3,
 5045: 3,
 5047: 3,
 5049: 3,
 5051: 3,
 5053: 3,
 5061: 3,
 5063: 3,
 5071: 3,
 5083: 3,
 5085: 3,
 5087: 3,
 5089: 3,
 5091: 3,
 5097: 3,
 5101: 3,
 5105: 3,
 5109: 3,
 5113: 3,
 5115: 3,
 5117: 3,
 5125: 3,
 5127: 3,
 5129: 3,
 5131: 3,
 5133: 3,
 5135: 3,
 5137: 3,
 5139: 3,
 5141: 3,


In [13]:
# Make a list of 2016 bellwethers

bellwethers_2016 = []
for x in county_wins.items():
    if x[1] == 5:
        bellwethers_2016.append(x)

In [14]:
len(bellwethers_2016)

58

## Import 2020 Data
https://www.kaggle.com/unanimad/us-election-2020?select=president_county_candidate.csv

In [15]:
df2 = pd.read_csv('data/raw_data/election_data_2020.csv')
df2 = df2[(df2['party'] == 'DEM') | (df2['party'] == 'REP')]
df2 = df2[df2['won'] == True]
df2 = df2[df2['county'] != 'Ward 2']
df2 = df2[df2['county'] != 'Ward 3']
df2 = df2[df2['county'] != 'Ward 4']
df2 = df2[df2['county'] != 'Ward 5']
df2 = df2[df2['county'] != 'Ward 6']
df2 = df2[df2['county'] != 'Ward 7']
df2 = df2[df2['county'] != 'Ward 8']
df2 = df2[df2['state'] != 'Hawaii']
df2 = df2[df2['state'] != 'Alaska']

In [16]:
df2.head()

state                county     candidate party  \
0               Delaware           Kent County     Joe Biden   DEM   
4               Delaware     New Castle County     Joe Biden   DEM   
8               Delaware         Sussex County  Donald Trump   REP   
12  District of Columbia  District of Columbia     Joe Biden   DEM   
68               Florida        Alachua County     Joe Biden   DEM   

    total_votes   won  
0         44552  True  
4        195034  True  
8         71230  True  
12        39041  True  
68        89704  True

In [17]:
df2.shape

(4581, 6)

In [18]:
df2['election_winner'] = 'D'

In [19]:
df2.head()

state                county     candidate party  \
0               Delaware           Kent County     Joe Biden   DEM   
4               Delaware     New Castle County     Joe Biden   DEM   
8               Delaware         Sussex County  Donald Trump   REP   
12  District of Columbia  District of Columbia     Joe Biden   DEM   
68               Florida        Alachua County     Joe Biden   DEM   

    total_votes   won election_winner  
0         44552  True               D  
4        195034  True               D  
8         71230  True               D  
12        39041  True               D  
68        89704  True               D

In [20]:
# Change won's values from True/False to D/R

for i, v in enumerate(df2['party']):
    if v == 'REP':
        df2.iloc[i,5] = 'R'
    elif v == 'DEM':
        df2.iloc[i,5] = 'D'

In [21]:
df2.head()

state                county     candidate party  \
0               Delaware           Kent County     Joe Biden   DEM   
4               Delaware     New Castle County     Joe Biden   DEM   
8               Delaware         Sussex County  Donald Trump   REP   
12  District of Columbia  District of Columbia     Joe Biden   DEM   
68               Florida        Alachua County     Joe Biden   DEM   

    total_votes won election_winner  
0         44552   D               D  
4        195034   D               D  
8         71230   R               D  
12        39041   D               D  
68        89704   D               D

In [22]:
# Get rid of ' County' and ' Parish' in each county name, get rid of cities

cities = []

for a, b in enumerate(df2['county']):
    name = b
    if ' County' in name:
        name = list(name)
        new_name = []
        for y, x in enumerate(name):
            if name[y] == ' ' and name[y+1] == 'C' and name[y+2] == 'o' and name[y+3] == 'u':
                if name[y+4] == 'n' and name[y+5] == 't' and name[y+6] == 'y':
                    break
            new_name.append(x)
        name = ''.join(new_name)
    elif ' Parish' in name:
        name = list(name)
        new_name = []
        for y, x in enumerate(name):
            if name[y] == ' ' and name[y+1] == 'P' and name[y+2] == 'a' and name[y+3] == 'r':
                if name[y+4] == 'i' and name[y+5] == 's' and name[y+6] == 'h':
                    break
            new_name.append(x)
        name = ''.join(new_name)
    elif ' city' in name:
        cities.append(a)
    df2.iloc[a, 1] = name

df2 = df2.reset_index(drop=True)
df2 = df2.drop(cities)

In [23]:
# Add a column describing correct prediction of the 2020 election

df2['predicted'] = ''
for i, v in enumerate(df2['won']):
    if v == list(df2['election_winner'])[i]:
        df2.iloc[i, -1] = 1
    else:
        df2.iloc[i, -1] = 0

In [24]:
df2.head()

state                county     candidate party  \
0              Delaware                  Kent     Joe Biden   DEM   
1              Delaware            New Castle     Joe Biden   DEM   
2              Delaware                Sussex  Donald Trump   REP   
3  District of Columbia  District of Columbia     Joe Biden   DEM   
4               Florida               Alachua     Joe Biden   DEM   

   total_votes won election_winner predicted  
0        44552   D               D         1  
1       195034   D               D         1  
2        71230   R               D         0  
3        39041   D               D         1  
4        89704   D               D         1

In [25]:
# Function to retrieve the fips value of a county from df

def get_fips(x):
    # where x is a tuple (state, county) in df
    returned_iloc = False
    for i, v in enumerate(df['county']):
        if x[1] == v and x[0] == list(df['state'])[i]:
            returned_iloc = True
            return int(list(df['FIPS'])[i])
            break
    if returned_iloc == False:
        return 'Not in dataset'
    
# Function to retrieve location of county in df2

def riloc(x):
    # where x is a tuple (state, county) in df2
    returned_iloc = False
    for i, v in enumerate(df2['county']):
        if x[1] == v and x[0] == list(df2['state'])[i]:
            returned_iloc = True
            return i
            break
    if returned_iloc == False:
        return 'Not in dataset'

In [26]:
# Add fips to df2

df2['fips'] = ''
for a, b in enumerate(df2['county']):
    for i, v in enumerate(df['county']):
        if v == b and list(df['state'])[i] == list(df2['state'])[a]:
            df2.iloc[a, -1] = int(list(df['FIPS'])[i])
            break

In [27]:
df2.head()

state                county     candidate party  \
0              Delaware                  Kent     Joe Biden   DEM   
1              Delaware            New Castle     Joe Biden   DEM   
2              Delaware                Sussex  Donald Trump   REP   
3  District of Columbia  District of Columbia     Joe Biden   DEM   
4               Florida               Alachua     Joe Biden   DEM   

   total_votes won election_winner predicted   fips  
0        44552   D               D         1  10001  
1       195034   D               D         1  10003  
2        71230   R               D         0  10005  
3        39041   D               D         1  11001  
4        89704   D               D         1  12001

In [28]:
# Manually add in the rest of the data due to inconsistencies between the datasets
# ex: in df2 a county is named DeSoto, Florida while in df it is named Desoto, Florida

df2.iloc[riloc(('Florida', 'DeSoto')), 8] = get_fips(('Florida', 'Desoto'))
df2 = df2[df2['county'] != 'Cook Suburbs']
df2.iloc[riloc(('Louisiana', 'LaSalle')), 8] = get_fips(('Louisiana', 'La Salle'))
df2.iloc[riloc(('Minnesota', 'Lac qui Parle')), 8] = get_fips(('Minnesota', 'Lac Qui Parle'))
df2.iloc[riloc(('Minnesota', 'St. Louis')), 8] = get_fips(('Minnesota', 'Saint Louis'))
df2.iloc[riloc(('Missouri', 'St. Louis')), 8] = get_fips(('Missouri', 'St. Louis County'))
df2.iloc[riloc(('New Mexico', 'Do�a Ana')), 8] = get_fips(('New Mexico', 'Dona Ana'))
df2.iloc[riloc(('Texas', 'DeWitt')), 8] = get_fips(('Texas', 'Dewitt'))

In [29]:
# No need for this data

df2 = df2[df2['state'] != 'Maine']
df2 = df2[df2['state'] != 'Maine']
df2 = df2[df2['state'] != 'Massachusetts']
df2 = df2[df2['state'] != 'New Hampshire']
df2 = df2[df2['state'] != 'Rhode Island']
df2 = df2[df2['state'] != 'Vermont']
df2 = df2[df2['state'] != 'Connecticut']

In [30]:
# Since some states in df2 were split by townships/precincts, we manually add county-level data from our source
# source: https://www.nytimes.com/interactive/2020/11/03/us/elections/results-president.html

values = []

# Maine - voted REP
for x in ['Aroostook', 'Somerset', 'Piscataquis', 'Penobscot', 'Washington', 'Oxford', 'Franklin', 'Androscoggin']:
    values.append(['Maine', x, 'Donald Trump', 'REP', '', 'R', 'D', 0, get_fips(('Maine', x))])
# Maine - voted DEM
for x in ['York', 'Cumberland', 'Sagadahoc', 'Kennebec', 'Lincoln', 'Waldo', 'Knox', 'Hancock']:
    values.append(['Maine', x, 'Joe Biden', 'DEM', '', 'D', 'D', 1, get_fips(('Maine', x))])

# Massachusetts - voted DEM
for x in ['Berkshire', 'Franklin', 'Hampshire', 'Hampden', 'Worcester', 'Middlesex', 'Essex', 'Suffolk', 
          'Norfolk', 'Plymouth', 'Bristol', 'Barnstable', 'Dukes', 'Nantucket']:
    values.append(['Massachusetts', x, 'Joe Biden', 'DEM', '', 'D', 'D', 1, get_fips(('Massachusetts', x))])

# New Hampshire - voted REP
for x in ['Coos', 'Belknap']:
    values.append(['New Hampshire', x, 'Donald Trump', 'REP', '', 'R', 'D', 0, get_fips(('New Hampshire', x))])
# New Hampshire - voted DEM
for x in ['Grafton', 'Carroll', 'Sullivan', 'Strafford', 'Merrimack', 'Rockingham', 'Hillsborough', 'Cheshire']:
    values.append(['New Hampshire', x, 'Joe Biden', 'DEM', '', 'D', 'D', 1, get_fips(('New Hampshire', x))])

# Rhode Island - voted DEM
for x in ['Providence', 'Kent', 'Washington', 'Bristol', 'Newport']:
    values.append(['Rhode Island', x, 'Joe Biden', 'DEM', '', 'D', 'D', 1, get_fips(('Rhode Island', x))])
    
# Vermont - voted REP
for x in ['Essex']:
    values.append(['Vermont', x, 'Donald Trump', 'REP', '', 'R', 'D', 0, get_fips(('Vermont', x))])
# Vermont - voted DEM
for x in ['Orleans', 'Franklin', 'Grand Isle', 'Chittenden', 'Caledonia', 'Lamoille', 'Washington', 'Addison', 
          'Orange', 'Windsor', 'Rutland', 'Bennington', 'Windham']:
    values.append(['Vermont', x, 'Joe Biden', 'DEM', '', 'D', 'D', 1, get_fips(('Vermont', x))])
    
# Connecticut - voted REP
for x in ['Windham', 'Litchfield']:
    values.append(['Connecticut', x, 'Donald Trump', 'REP', '', 'R', 'D', 0, get_fips(('Connecticut', x))])
# Connecticut - voted DEM
for x in ['Tolland', 'Hartford', 'New London', 'Middlesex', 'New Haven', 'Fairfield']:
    values.append(['Connecticut', x, 'Joe Biden', 'DEM', '', 'D', 'D', 1, get_fips(('Connecticut', x))])

df2 = df2.append(pd.DataFrame(values, columns=df2.columns), ignore_index=True)

In [31]:
df2.tail(75)

state        county     candidate party total_votes won  \
2993        Arizona       Cochise  Donald Trump   REP       35557   R   
2994        Arizona      Coconino     Joe Biden   DEM       44698   D   
2995        Arizona          Gila  Donald Trump   REP       18377   R   
2996        Arizona        Graham  Donald Trump   REP       10749   R   
2997        Arizona      Greenlee  Donald Trump   REP        2433   R   
2998        Arizona        La Paz  Donald Trump   REP        5129   R   
2999        Arizona      Maricopa     Joe Biden   DEM     1040774   D   
3000        Arizona        Mohave  Donald Trump   REP       78535   R   
3001          Maine     Aroostook  Donald Trump   REP               R   
3002          Maine      Somerset  Donald Trump   REP               R   
3003          Maine   Piscataquis  Donald Trump   REP               R   
3004          Maine     Penobscot  Donald Trump   REP               R   
3005          Maine    Washington  Donald Trump   REP               R   
3006          Maine        Oxford  Donald Trump   REP               R   
3007          Maine      Franklin  Donald Trump   REP               R   
3008          Maine  Androscoggin  Donald Trump   REP               R   
3009          Maine          York     Joe Biden   DEM               D   
3010          Maine    Cumberland     Joe Biden   DEM               D   
3011          Maine     Sagadahoc     Joe Biden   DEM               D   
3012          Maine      Kennebec     Joe Biden   DEM               D   
3013          Maine       Lincoln     Joe Biden   DEM               D   
3014          Maine         Waldo     Joe Biden   DEM               D   
3015          Maine          Knox     Joe Biden   DEM               D   
3016          Maine       Hancock     Joe Biden   DEM               D   
3017  Massachusetts     Berkshire     Joe Biden   DEM               D   
3018  Massachusetts      Franklin     Joe Biden   DEM               D   
3019  Massachusetts     Hampshire     Joe Biden   DEM               D   
3020  Massachusetts       Hampden     Joe Biden   DEM               D   
3021  Massachusetts     Worcester     Joe Biden   DEM               D   
3022  Massachusetts     Middlesex     Joe Biden   DEM               D   
3023  Massachusetts         Essex     Joe Biden   DEM               D   
3024  Massachusetts       Suffolk     Joe Biden   DEM               D   
3025  Massachusetts       Norfolk     Joe Biden   DEM               D   
3026  Massachusetts      Plymouth     Joe Biden   DEM               D   
3027  Massachusetts       Bristol     Joe Biden   DEM               D   
3028  Massachusetts    Barnstable     Joe Biden   DEM               D   
3029  Massachusetts         Dukes     Joe Biden   DEM               D   
3030  Massachusetts     Nantucket     Joe Biden   DEM               D   
3031  New Hampshire          Coos  Donald Trump   REP               R   
3032  New Hampshire       Belknap  Donald Trump   REP               R   
3033  New Hampshire       Grafton     Joe Biden   DEM               D   
3034  New Hampshire       Carroll     Joe Biden   DEM               D   
3035  New Hampshire      Sullivan     Joe Biden   DEM               D   
3036  New Hampshire     Strafford     Joe Biden   DEM               D   
3037  New Hampshire     Merrimack     Joe Biden   DEM               D   
3038  New Hampshire    Rockingham     Joe Biden   DEM               D   
3039  New Hampshire  Hillsborough     Joe Biden   DEM               D   
3040  New Hampshire      Cheshire     Joe Biden   DEM               D   
3041   Rhode Island    Providence     Joe Biden   DEM               D   
3042   Rhode Island          Kent     Joe Biden   DEM               D   
3043   Rhode Island    Washington     Joe Biden   DEM               D   
3044   Rhode Island       Bristol     Joe Biden   DEM               D   
3045   Rhode Island       Newport     Joe Biden   DEM               D   
3046        Vermont         Essex  Donald Trump   REP               R   

In [32]:
df2[df2['fips'] == ''].shape

(0, 9)

In [33]:
# Update county_wins
for i, v in enumerate(df2['fips']):
    if list(df2['won'])[i] == list(df2['election_winner'])[i]:
        if v not in county_wins.keys():
            county_wins[v] = 1
        elif v in county_wins.keys():
            county_wins[v] += 1

In [34]:
# Make a list of 2020 bellwethers

bellwethers_2020 = []
for x in county_wins.items():
    if x[1] > 5:
        bellwethers_2020.append(x)

In [35]:
bellwethers_2020

[(10001, 6),
 (27027, 6),
 (30005, 6),
 (33011, 6),
 (36031, 6),
 (36091, 6),
 (53009, 6),
 (55029, 6)]

In [36]:
len(bellwethers_2020)

8

## Make a List of Polarized Counties

In [37]:
df.head()

year    state state_po   county    FIPS     office       candidate  \
1   2000  Alabama       AL  Autauga  1001.0  President  George W. Bush   
5   2000  Alabama       AL  Baldwin  1003.0  President  George W. Bush   
8   2000  Alabama       AL  Barbour  1005.0  President         Al Gore   
13  2000  Alabama       AL     Bibb  1007.0  President  George W. Bush   
17  2000  Alabama       AL   Blount  1009.0  President  George W. Bush   

         party  candidatevotes  totalvotes   version  percentage winner  \
1   republican         11993.0       17208  20191203    0.696943      R   
5   republican         40872.0       56480  20191203    0.723654      R   
8     democrat          5188.0       10395  20191203    0.499086      D   
13  republican          4273.0        7101  20191203    0.601746      R   
17  republican         12667.0       17973  20191203    0.704779      R   

   election_winner  
1                R  
5                R  
8                R  
13               R  
17               R

In [38]:
df2.head()

state                county     candidate party total_votes  \
0              Delaware                  Kent     Joe Biden   DEM       44552   
1              Delaware            New Castle     Joe Biden   DEM      195034   
2              Delaware                Sussex  Donald Trump   REP       71230   
3  District of Columbia  District of Columbia     Joe Biden   DEM       39041   
4               Florida               Alachua     Joe Biden   DEM       89704   

  won election_winner predicted   fips  
0   D               D         1  10001  
1   D               D         1  10003  
2   R               D         0  10005  
3   D               D         1  11001  
4   D               D         1  12001

In [39]:
# Sort through 2000-2020 data

count = {}
for i, v in enumerate(df['FIPS']):
    if v in count.keys() and list(df['winner'])[i] == 'R':
        count[v] += 1
    elif v not in count.keys():
        if list(df['winner'])[i] == 'R':
            count[v] = 1
        if list(df['winner'])[i] == 'D':
            count[v] = 0
for i, v in enumerate(df2['fips']):
    if v in count.keys() and list(df2['won'])[i] == 'R':
        count[v] += 1
    elif v not in count.keys():
        if list(df2['won'])[i] == 'R':
            count[v] = 1
        if list(df2['won'])[i] == 'D':
            count[v] = 0

In [40]:
polar_counties_fips = []
for item in count.items():
    if item[1] == 0 or item[1] == 6:
        polar_counties_fips.append(item[0])

In [41]:
polar_counties_names = []
for x in polar_counties_fips:
    for i, v in enumerate(df['FIPS']):
        if v == x:
            polar_counties_names.append((list(df['state'])[i], list(df['county'])[i]))
            break

In [42]:
len(polar_counties_names)

2402

In [43]:
polar_counties_fips = pd.DataFrame(polar_counties_fips)
polar_counties_fips.head()

0
0  1001.0
1  1003.0
2  1007.0
3  1009.0
4  1011.0

In [44]:
polar_counties_fips.to_csv('data/raw_data/polar_counties_fips.csv') # save this data for future use

In [45]:
polar_counties_names = pd.DataFrame(polar_counties_names)
polar_counties_names.head()

0        1
0  Alabama  Autauga
1  Alabama  Baldwin
2  Alabama     Bibb
3  Alabama   Blount
4  Alabama  Bullock

In [46]:
polar_counties_names.to_csv('data/raw_data/polar_counties_names.csv') # save this data for future use

## Import Poverty Rate Data
Poverty Rates: https://www.census.gov/library/visualizations/time-series/demo/census-poverty-tool.html

In [47]:
p_df = pd.read_csv("data/raw_data/poverty_rate.csv", engine = "python")
p_df = p_df.drop(columns=['Number'])
p_df.head()

FIPS    State       County  Poverty Rate 1960  Poverty Rate 1970  \
0     1  Alabama  State Total               42.5               25.4   
1  1001  Alabama    Autauga                 53.1               23.7   
2  1003  Alabama    Baldwin                 41.8               22.6   
3  1005  Alabama     Barbour                65.7               43.7   
4  1007  Alabama        Bibb                56.8               38.2   

   Poverty Rate 1980  Poverty Rate 1990  Poverty Rate 2000  Poverty Rate 2010  \
0               18.9               18.3               16.1               18.1   
1               16.6               15.7               10.9               11.6   
2               16.2               14.3               10.1               13.3   
3               30.8               25.2               26.8               26.1   
4               21.2               21.2               20.6               16.5   

  Population 1960 Population 1970 Population 1980 Population 1990  \
0       3,235,402       3,368,487       3,813,014       3,945,798   
1          18,454          24,313          32,107          34,009   
2          49,722          58,776          77,726          97,227   
3          24,975          22,424          24,533          24,785   
4          14,775          13,761          15,520          16,304   

  Population 2000 Population 2010 Persons in Poverty 1960  \
0       4,334,919       4,662,094               1,373,985   
1          43,377          54,255                   9,807   
2         138,148         180,970                  20,773   
3          26,239          24,530                  16,395   
4          19,852          21,794                   8,386   

  Persons in Poverty 1970 Persons in Poverty 1980 Persons in Poverty 1990  \
0                 857,248                 719,905                 723,614   
1                   5,758                   5,337                   5,324   
2                  13,304                  12,616                  13,947   
3                   9,788                   7,559                   6,244   
4                   5,256                   3,296                   3,454   

  Persons in Poverty 2000 Persons in Poverty 2010  
0                 698,097                 842,292  
1                   4,738                   6,267  
2                  14,018                  24,002  
3                   7,032                   6,395  
4                   4,091                   3,594

In [48]:
p_df.columns

Index(['FIPS', 'State', 'County', 'Poverty Rate 1960', 'Poverty Rate 1970',
       'Poverty Rate 1980', 'Poverty Rate 1990', 'Poverty Rate 2000',
       'Poverty Rate 2010', 'Population 1960', 'Population 1970',
       'Population 1980', 'Population 1990', 'Population 2000',
       'Population 2010', 'Persons in Poverty 1960', 'Persons in Poverty 1970',
       'Persons in Poverty 1980', 'Persons in Poverty 1990',
       'Persons in Poverty 2000', 'Persons in Poverty 2010'],
      dtype='object')

In [49]:
p_df = p_df[p_df["County"] != "State Total"]
p_df = p_df[p_df["State"] != "Hawaii"]
p_df = p_df[p_df["State"] != "Alaska"]

In [50]:
# Remove random spaces

for i, v in enumerate(p_df['County']):
    v = list(v)
    for z in range(len(v)):
        if v[-1] == ' ':
            v.pop(-1)
        elif v[-1] != ' ':
            break
    p_df.iloc[i, 2] = ''.join(v)

In [51]:
p_df.shape # 3108 mainland counties

(3109, 21)

In [52]:
p_df = p_df[p_df['FIPS'] != 51515] # extra county

In [53]:
# Remove commas from numerical values

columns = ['Population 1960', 'Population 1970',
       'Population 1980', 'Population 1990', 'Population 2000',
       'Population 2010', 'Persons in Poverty 1960', 'Persons in Poverty 1970',
       'Persons in Poverty 1980', 'Persons in Poverty 1990',
       'Persons in Poverty 2000', 'Persons in Poverty 2010']

for x in range(len(columns)):
    for i, v in enumerate(p_df[columns[x]]):
        if type(v) == str and ',' in v:
            v = int(''.join(v.split(',')))
            p_df.iloc[i, x + 9] = v

## Import Unemployment Rate and Median Household Income Data
Unemployment Data: https://data.ers.usda.gov/reports.aspx?ID=17828

In [54]:
u_df = pd.read_csv('data/raw_data/unemployment_rate.csv')
u_df.drop([0, 1, 2])

# Rename & Drop Columns

n_df = pd.DataFrame()
for i, v in enumerate(u_df.columns):
    n_df[str(u_df.iloc[3, i])] = u_df.iloc[4:, i]
u_df = n_df

In [55]:
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", 
               "Connecticut", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", 
               "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", 
               "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", 
               "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", 
               "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", 
               "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", 
               "West Virginia", "Wyoming"]

In [56]:
for state in state_names:
    u_df = u_df[u_df["area_name"] != state]
u_df = u_df[u_df['Stabr'] != 'AK']
u_df = u_df[u_df['Stabr'] != 'HI']
u_df = u_df[u_df['Stabr'] != 'US']
u_df = u_df[u_df['Stabr'] != 'PR']
u_df = u_df[u_df['fips_txt'] != '11001']

In [57]:
u_df.shape # right amount of counties

(3108, 88)

In [58]:
# Check if all u_df fips in p_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

u_fips = []
for i, v in enumerate(u_df['fips_txt']):
    u_fips.append(int(v))
    
for i, v in enumerate(u_fips):
    if v not in ideal_fips:
        print(v)

11000
46102


In [59]:
u_df.loc[(u_df['fips_txt'] == '11000')|(u_df['fips_txt'] == '46102')]

fips_txt Stabr                 area_name Rural_urban_continuum_code_2013  \
335     11000    DC      District of Columbia                             NaN   
2461    46102    SD  Oglala Lakota County, SD                               6   

     Urban_influence_code_2013 Metro_2013 Civilian_labor_force_2000  \
335                        NaN        NaN                   310,469   
2461                         6          0                     2,845   

     Employed_2000 Unemployed_2000 Unemployment_rate_2000  \
335        293,086          17,383                    5.6   
2461         2,603             242                    8.5   

     Civilian_labor_force_2001 Employed_2001 Unemployed_2001  \
335                    310,515       290,811          19,704   
2461                     3,666         3,376             290   

     Unemployment_rate_2001 Civilian_labor_force_2002 Employed_2002  \
335                     6.3                   306,099       286,496   
2461                    7.9                     3,779         3,504   

     Unemployed_2002 Unemployment_rate_2002 Civilian_labor_force_2003  \
335           19,603                    6.4                   306,422   
2461             275                    7.3                     3,922   

     Employed_2003 Unemployed_2003 Unemployment_rate_2003  \
335        285,454          20,968                    6.8   
2461         3,554             368                    9.4   

     Civilian_labor_force_2004 Employed_2004 Unemployed_2004  \
335                    312,027       287,683          24,344   
2461                     3,902         3,543             359   

     Unemployment_rate_2004 Civilian_labor_force_2005 Employed_2005  \
335                     7.8                   315,616       295,484   
2461                    9.2                     3,963         3,530   

     Unemployed_2005 Unemployment_rate_2005 Civilian_labor_force_2006  \
335           20,132                    6.4                   316,083   
2461             433                   10.9                     4,036   

     Employed_2006 Unemployed_2006 Unemployment_rate_2006  \
335        297,786          18,297                    5.8   
2461         3,705             331                    8.2   

     Civilian_labor_force_2007 Employed_2007 Unemployed_2007  \
335                    322,237       304,426          17,811   
2461                     3,798         3,501             297   

     Unemployment_rate_2007 Civilian_labor_force_2008 Employed_2008  \
335                     5.5                   330,544       309,192   
2461                    7.8                     3,788         3,469   

     Unemployed_2008 Unemployment_rate_2008 Civilian_labor_force_2009  \
335           21,352                    6.5                   335,672   
2461             319                    8.4                     3,995   

     Employed_2009 Unemployed_2009 Unemployment_rate_2009  \
335        304,500          31,172                    9.3   
2461         3,560             435                   10.9   

     Civilian_labor_force_2010 Employed_2010 Unemployed_2010  \
335                    346,065       313,508          32,557   
2461                     3,715         3,234             481   

     Unemployment_rate_2010 Civilian_labor_force_2011 Employed_2011  \
335                     9.4                   350,778       315,171   
2461                   12.9                     3,726         3,198   

     Unemployed_2011 Unemployment_rate_2011 Civilian_labor_force_2012  \
335           35,607                   10.2                   364,989   
2461             528                   14.2                     3,756   

     Employed_2012 Unemployed_2012 Unemployment_rate_2012  \
335        331,984          33,005                      9   
2461         3,221             535                   14.2   

     Civilian_labor_force_2013 Employed_2013 Unemployed_2013  \
335                    374,126       342,351          3

In [60]:
u_df.iloc[290, 0] = 11

In [61]:
# Check if all p_df fips in u_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

u_fips = []
for i, v in enumerate(u_df['fips_txt']):
    u_fips.append(int(v))
    
for i, v in enumerate(ideal_fips):
    if v not in u_fips:
        print(v)

46113


In [62]:
p_df.loc[(p_df['FIPS'] == 11)|(p_df['FIPS'] == 46113)]

FIPS                 State   County  Poverty Rate 1960  \
327      11  District of Columbia    Total               22.2   
2458  46113          South Dakota  Shannon               63.5   

      Poverty Rate 1970  Poverty Rate 1980  Poverty Rate 1990  \
327                17.0               18.6               16.9   
2458               45.7               44.7               63.1   

      Poverty Rate 2000  Poverty Rate 2010 Population 1960 Population 1970  \
327                20.2               18.5          724518          724306   
2458               52.3               49.5            5748            8059   

     Population 1980 Population 1990 Population 2000 Population 2010  \
327           610454          570826          541657          572108   
2458           11187            9693           12204           13449   

     Persons in Poverty 1960 Persons in Poverty 1970 Persons in Poverty 1980  \
327                   160960                  123109                  113356   
2458                    3649                    3684                    4998   

     Persons in Poverty 1990 Persons in Poverty 2000 Persons in Poverty 2010  
327                    96278                  109500                  105606  
2458                    6118                    6385                    6651

In [63]:
p_df.iloc[2383, 2] = 'Oglala Lakota'
p_df.iloc[2383, 0] = 46102

In [64]:
# Remove commas from numerical values

columns = ['Civilian_labor_force_2000',
       'Employed_2000', 'Unemployed_2000', 'Unemployment_rate_2000',
       'Civilian_labor_force_2001', 'Employed_2001', 'Unemployed_2001',
       'Unemployment_rate_2001', 'Civilian_labor_force_2002', 'Employed_2002',
       'Unemployed_2002', 'Unemployment_rate_2002',
       'Civilian_labor_force_2003', 'Employed_2003', 'Unemployed_2003',
       'Unemployment_rate_2003', 'Civilian_labor_force_2004', 'Employed_2004',
       'Unemployed_2004', 'Unemployment_rate_2004',
       'Civilian_labor_force_2005', 'Employed_2005', 'Unemployed_2005',
       'Unemployment_rate_2005', 'Civilian_labor_force_2006', 'Employed_2006',
       'Unemployed_2006', 'Unemployment_rate_2006',
       'Civilian_labor_force_2007', 'Employed_2007', 'Unemployed_2007',
       'Unemployment_rate_2007', 'Civilian_labor_force_2008', 'Employed_2008',
       'Unemployed_2008', 'Unemployment_rate_2008',
       'Civilian_labor_force_2009', 'Employed_2009', 'Unemployed_2009',
       'Unemployment_rate_2009', 'Civilian_labor_force_2010', 'Employed_2010',
       'Unemployed_2010', 'Unemployment_rate_2010',
       'Civilian_labor_force_2011', 'Employed_2011', 'Unemployed_2011',
       'Unemployment_rate_2011', 'Civilian_labor_force_2012', 'Employed_2012',
       'Unemployed_2012', 'Unemployment_rate_2012',
       'Civilian_labor_force_2013', 'Employed_2013', 'Unemployed_2013',
       'Unemployment_rate_2013', 'Civilian_labor_force_2014', 'Employed_2014',
       'Unemployed_2014', 'Unemployment_rate_2014',
       'Civilian_labor_force_2015', 'Employed_2015', 'Unemployed_2015',
       'Unemployment_rate_2015', 'Civilian_labor_force_2016', 'Employed_2016',
       'Unemployed_2016', 'Unemployment_rate_2016',
       'Civilian_labor_force_2017', 'Employed_2017', 'Unemployed_2017',
       'Unemployment_rate_2017', 'Civilian_labor_force_2018', 'Employed_2018',
       'Unemployed_2018', 'Unemployment_rate_2018',
       'Civilian_labor_force_2019', 'Employed_2019', 'Unemployed_2019',
       'Unemployment_rate_2019', 'Median_Household_Income_2019',
       'Med_HH_Income_Percent_of_State_Total_2019']

for x in range(len(columns)):
    for i, v in enumerate(u_df[columns[x]]):
        if type(v) == str and ',' in v:
            v = int(''.join(v.split(',')))
            u_df.iloc[i, x + 6] = v

## Import Education Data
Education Data: https://data.ers.usda.gov/reports.aspx?ID=17829

In [65]:
e_df = pd.read_csv('data/raw_data/education_rate.csv', engine='python')

In [66]:
for state in state_names:
    e_df = e_df[e_df["Area name"] != state]
e_df = e_df[e_df['State'] != 'AK']
e_df = e_df[e_df['State'] != 'HI']
e_df = e_df[e_df['State'] != 'US']
e_df = e_df[e_df['State'] != 'PR']
e_df = e_df[e_df['FIPS Code'] != 11001]

In [67]:
e_df.shape # 3108 mainland counties

(3112, 47)

In [68]:
# Check if all e_df fips in p_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

e_fips = []
for i, v in enumerate(e_df['FIPS Code']):
    e_fips.append(int(v))
    
for i, v in enumerate(e_fips):
    if v not in ideal_fips:
        print(v)

11000
22000
30113
51515
51560


In [69]:
e_df.loc[(e_df['FIPS Code'] == 22000)|(e_df['FIPS Code'] == 30113)|(e_df['FIPS Code'] == 51515)|(e_df['FIPS Code'] == 51560)]

FIPS Code State                  Area name  \
1139      22000    LA                   Lousiana   
1689      30113    MT  Yellowstone National Park   
2972      51515    VA               Bedford city   
2977      51560    VA         Clifton Forge city   

      2003 Rural-urban Continuum Code  2003 Urban Influence Code  \
1139                              NaN                        NaN   
1689                              7.0                        9.0   
2972                              3.0                        2.0   
2977                              6.0                        6.0   

      2013 Rural-urban Continuum Code  2013 Urban Influence Code  \
1139                              NaN                        NaN   
1689                              NaN                        NaN   
2972                              2.0                        2.0   
2977                              NaN                        NaN   

     Less than a high school diploma, 1970 High school diploma only, 1970  \
1139                             1,046,551                        446,400   
1689                                     6                             14   
2972                                 2,106                            878   
2977                                 1,505                          1,220   

     Some college (1-3 years), 1970 Four years of college or higher, 1970  \
1139                        153,171                               163,792   
1689                              6                                     0   
2972                            388                                   319   
2977                            307                                   236   

      Percent of adults with less than a high school diploma, 1970  \
1139                                               57.8              
1689                                               23.1              
2972                                               57.1              
2977                                               46.1              

      Percent of adults with a high school diploma only, 1970  \
1139                                               24.7         
1689                                               53.8         
2972                                               23.8         
2977                                               37.3         

      Percent of adults completing some college (1-3 years), 1970  \
1139                                                8.5             
1689                                               23.1             
2972                                               10.5             
2977                                                9.4             

      Percent of adults completing four years of college or higher, 1970  \
1139                                                9.0                    
1689                                                0.0                    
2972                                                8.6                    
2977                                                7.2                    

     Less than a high school diploma, 1980 High school diploma only, 1980  \
1139                               965,657                        705,808   
1689                                    42                             35   
2972                                 1,925                          1,194   
2977                                 1,569                            977   

     Some college (1-3 years), 1980 Four years of college or higher, 1980  \
1139                        292,793                               317,223   
1689                             36                                    56   
2972                            410                                   535   
2977                            431                                   341   

      Percent of adults with less than a high school diploma, 1980  \
1139                                         

In [70]:
e_df = e_df[e_df['Area name'] != 'Lousiana'] # typo
e_df = e_df[e_df['Area name'] != 'Yellowstone National Park'] # former county
e_df = e_df[e_df['Area name'] != 'Bedford city'] # apart of Roanoke county
e_df = e_df[e_df['Area name'] != 'Clifton Forge city'] # apart of Alleghany county

In [71]:
# Check if all p_df fips in e_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

e_fips = []
for i, v in enumerate(e_df['FIPS Code']):
    e_fips.append(int(v))
    
for i, v in enumerate(ideal_fips):
    if v not in e_fips:
        print(v)

11


In [72]:
e_df.iloc[290, 0] = 11

In [73]:
e_df.shape

(3108, 47)

In [74]:
e_df.columns

Index(['FIPS Code', 'State', 'Area name', '2003 Rural-urban Continuum Code',
       '2003 Urban Influence Code', '2013 Rural-urban Continuum Code',
       '2013 Urban Influence Code', 'Less than a high school diploma, 1970',
       'High school diploma only, 1970', 'Some college (1-3 years), 1970',
       'Four years of college or higher, 1970',
       'Percent of adults with less than a high school diploma, 1970',
       'Percent of adults with a high school diploma only, 1970',
       'Percent of adults completing some college (1-3 years), 1970',
       'Percent of adults completing four years of college or higher, 1970',
       'Less than a high school diploma, 1980',
       'High school diploma only, 1980', 'Some college (1-3 years), 1980',
       'Four years of college or higher, 1980',
       'Percent of adults with less than a high school diploma, 1980',
       'Percent of adults with a high school diploma only, 1980',
       'Percent of adults completing some college (1-3 years

In [75]:
# Remove commas from numerical values

columns = ['Less than a high school diploma, 1970',
       'High school diploma only, 1970', 'Some college (1-3 years), 1970',
       'Four years of college or higher, 1970',
       'Percent of adults with less than a high school diploma, 1970',
       'Percent of adults with a high school diploma only, 1970',
       'Percent of adults completing some college (1-3 years), 1970',
       'Percent of adults completing four years of college or higher, 1970',
       'Less than a high school diploma, 1980',
       'High school diploma only, 1980', 'Some college (1-3 years), 1980',
       'Four years of college or higher, 1980',
       'Percent of adults with less than a high school diploma, 1980',
       'Percent of adults with a high school diploma only, 1980',
       'Percent of adults completing some college (1-3 years), 1980',
       'Percent of adults completing four years of college or higher, 1980',
       'Less than a high school diploma, 1990',
       'High school diploma only, 1990',
       "Some college or associate's degree, 1990",
       "Bachelor's degree or higher, 1990",
       'Percent of adults with less than a high school diploma, 1990',
       'Percent of adults with a high school diploma only, 1990',
       "Percent of adults completing some college or associate's degree, 1990",
       "Percent of adults with a bachelor's degree or higher, 1990",
       'Less than a high school diploma, 2000',
       'High school diploma only, 2000',
       "Some college or associate's degree, 2000",
       "Bachelor's degree or higher, 2000",
       'Percent of adults with less than a high school diploma, 2000',
       'Percent of adults with a high school diploma only, 2000',
       "Percent of adults completing some college or associate's degree, 2000",
       "Percent of adults with a bachelor's degree or higher, 2000",
       'Less than a high school diploma, 2014-18',
       'High school diploma only, 2014-18',
       "Some college or associate's degree, 2014-18",
       "Bachelor's degree or higher, 2014-18"]

for x in range(len(columns)):
    for i, v in enumerate(e_df[columns[x]]):
        if type(v) == str and ',' in v:
            v = int(''.join(v.split(',')))
            e_df.iloc[i, x + 7] = v

## Import Demographics Data
Demographics Data: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-detail.html

In [76]:
d_df = pd.read_csv('data/raw_data/demographics.csv', encoding='cp1252')

In [77]:
d_df.shape

(716376, 80)

In [78]:
# Drop columns

d_df1 = d_df
d_df1 = d_df1[(d_df.STNAME != 'Alaska')&(d_df.STNAME != 'Hawaii')]
d_df1 = d_df1[(d_df.YEAR == 1)&(d_df.AGEGRP == 0)]

<ipython-input-78-1f1de3d30a8d>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d_df1 = d_df1[(d_df.YEAR == 1)&(d_df.AGEGRP == 0)]


In [79]:
d_df1.shape

(3108, 80)

In [80]:
d_df1.head()

SUMLEV  STATE  COUNTY   STNAME         CTYNAME  YEAR  AGEGRP  TOT_POP  \
0        50      1       1  Alabama  Autauga County     1       0    54571   
228      50      1       3  Alabama  Baldwin County     1       0   182265   
456      50      1       5  Alabama  Barbour County     1       0    27457   
684      50      1       7  Alabama     Bibb County     1       0    22915   
912      50      1       9  Alabama   Blount County     1       0    57322   

     TOT_MALE  TOT_FEMALE  WA_MALE  WA_FEMALE  BA_MALE  BA_FEMALE  IA_MALE  \
0       26569       28002    21295      22002     4559       5130      119   
228     89196       93069    78334      81376     8396       8878      662   
456     14576       12881     7463       6524     6808       6133       93   
684     12301       10614     9055       8494     3079       1981       35   
912     28362       28960    27398      27970      420        397      178   

     IA_FEMALE  AA_MALE  AA_FEMALE  NA_MALE  NA_FEMALE  TOM_MALE  TOM_FEMALE  \
0          139      200        284       29         18       367         429   
228        677      540        829       72         49      1192        1260   
456         73       59         53       29         18       124          80   
684         33       13          9       28          5        91          92   
912        173       45         80       38         25       283         315   

     WAC_MALE  WAC_FEMALE  BAC_MALE  BAC_FEMALE  IAC_MALE  IAC_FEMALE  \
0       21633       22391      4704        5306       277         314   
228     79452       82576      8804        9327      1295        1346   
456      7546        6583      6898        6182       162         116   
684      9133        8567      3119        2028        84          76   
912     27670       28275       493         472       364         376   

     AAC_MALE  AAC_FEMALE  NAC_MALE  NAC_FEMALE  NH_MALE  NH_FEMALE  \
0         300         409        42          37    25875      27386   
228       783        1072       127          84    84771      89502   
456        67          57        33          23    13799      12271   
684        26          30        33           9    12003      10506   
912        81         134        45          31    25793      26903   

     NHWA_MALE  NHWA_FEMALE  NHBA_MALE  NHBA_FEMALE  NHIA_MALE  NHIA_FEMALE  \
0        20709        21485       4512         5091        103          115   
228      74314        78160       8268         8754        558          592   
456       6832         6017       6746         6084         31           31   
684       8798         8410       3065         1967         35           32   
912      24982        26009        368          363        134          151   

     NHAA_MALE  NHAA_FEMALE  NHNA_MALE  NHNA_FEMALE  NHTOM_MALE  NHTOM_FEMALE  \
0          194          280         13            9         344           406   
228        532          817         47           39        1052          1140   
456         59           53         16            8         115            78   
684         13            9          6            1          86            87   
912         44           76         11            7         254           297   

     NHWAC_MALE  NHWAC_FEMALE  NHBAC_MALE  NHBAC_FEMALE  NHIAC_MALE  \
0         21026         21853        4647          5258         251   
228       75299         79251        8648          9173        1106   
456        6909          6074        6832          6133          93   
684        8876          8480        3100          2010          83   
912       25228         26297         433           434         298   

     NHIAC_FEMALE  NHAAC_MALE  NHAAC_FEMALE  NHNAC_MALE  NHNAC_FEMALE  H_MALE  \
0             282         291           398          23            27     694   
228          1183         729          1024          82            58    4425   
456            72          66            57          19            13     777   
684    

In [81]:
d_df2 = d_df
d_df2 = d_df2[(d_df.STNAME != 'Alaska')&(d_df.STNAME != 'Hawaii')]
d_df2 = d_df2[(d_df.YEAR == 12)&(d_df.AGEGRP == 0)]

<ipython-input-81-8443ae9e0985>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d_df2 = d_df2[(d_df.YEAR == 12)&(d_df.AGEGRP == 0)]


In [82]:
d_df2.head()

SUMLEV  STATE  COUNTY   STNAME         CTYNAME  YEAR  AGEGRP  TOT_POP  \
209       50      1       1  Alabama  Autauga County    12       0    55869   
437       50      1       3  Alabama  Baldwin County    12       0   223234   
665       50      1       5  Alabama  Barbour County    12       0    24686   
893       50      1       7  Alabama     Bibb County    12       0    22394   
1121      50      1       9  Alabama   Blount County    12       0    57826   

      TOT_MALE  TOT_FEMALE  WA_MALE  WA_FEMALE  BA_MALE  BA_FEMALE  IA_MALE  \
209      27092       28777    20878      21729     5237       6000      121   
437     108247      114987    94810     100388     9486      10107      903   
665      13064       11622     6389       5745     6311       5595      103   
893      11929       10465     8766       8425     2941       1822       53   
1121     28472       29354    27258      28154      516        462      192   

      IA_FEMALE  AA_MALE  AA_FEMALE  NA_MALE  NA_FEMALE  TOM_MALE  TOM_FEMALE  \
209         145      286        370       32         26       538         507   
437         839      932       1448       74         80      2042        2125   
665          67       55         61       34         18       172         136   
893          50       23         25       22          4       124         139   
1121        178       85        100       42         25       379         435   

      WAC_MALE  WAC_FEMALE  BAC_MALE  BAC_FEMALE  IAC_MALE  IAC_FEMALE  \
209      21372       22191      5514        6270       274         301   
437      96730      102373     10346       11008      1818        1785   
665       6517        5863      6432        5685       182         115   
893       8869        8551      3010        1897       106         101   
1121     27608       28559       658         618       400         420   

      AAC_MALE  AAC_FEMALE  NAC_MALE  NAC_FEMALE  NH_MALE  NH_FEMALE  \
209        441         509        55          49    26208      27990   
437       1405        1946       180         187   102702     109998   
665         72          80        38          23    12435      11134   
893         49          48        28          12    11586      10185   
1121       147         165        60          37    25522      26722   

      NHWA_MALE  NHWA_FEMALE  NHBA_MALE  NHBA_FEMALE  NHIA_MALE  NHIA_FEMALE  \
209       20138        21077       5171         5927        105          138   
437       89845        95902       9308         9907        753          754   
665        5894         5341       6260         5547         52           43   
893        8482         8181       2912         1807         50           41   
1121      24494        25682        453          419        143          139   

      NHAA_MALE  NHAA_FEMALE  NHNA_MALE  NHNA_FEMALE  NHTOM_MALE  \
209         282          364         20           20         492   
437         911         1435         53           70        1832   
665          55           61         21           10         153   
893          21           25          5            1         116   
1121         73           90         14            7         345   

      NHTOM_FEMALE  NHWAC_MALE  NHWAC_FEMALE  NHBAC_MALE  NHBAC_FEMALE  \
209            464       20594         21504        5425          6177   
437           1930       91586         97727       10078         10727   
665            132        6008          5455        6369          5635   
893            130        8578          8298        2978          1878   
1121           385       24814         26043         582           560   

      NHIAC_MALE  NHIAC_FEMALE  NHAAC_MALE  NHAAC_FEMALE  NHNAC_MALE  \
209          234           274         426           490          39   
437         1554          1588        1336          1881         125   
665          121            89          71            80          24   
893          100            86          44            47          11

## Import COVID-19 Data
COVID-19 Data: https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/ (updated 3/6/2021)

In [83]:
ccc_df = pd.read_csv('data/raw_data/covid_confirmed_cases.csv')
cd_df = pd.read_csv('data/raw_data/covid_deaths.csv')

In [84]:
print(ccc_df.shape, cd_df.shape)

(3193, 414) (3193, 414)


In [85]:
ccc_df.head()

countyFIPS            County Name State  StateFIPS  2020-01-22  2020-01-23  \
0           0  Statewide Unallocated    AL          1           0           0   
1        1001        Autauga County     AL          1           0           0   
2        1003        Baldwin County     AL          1           0           0   
3        1005        Barbour County     AL          1           0           0   
4        1007           Bibb County     AL          1           0           0   

   2020-01-24  2020-01-25  2020-01-26  2020-01-27  2020-01-28  2020-01-29  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-01-30  2020-01-31  2020-02-01  2020-02-02  2020-02-03  2020-02-04  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-05  2020-02-06  2020-02-07  2020-02-08  2020-02-09  2020-02-10  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-11  2020-02-12  2020-02-13  2020-02-14  2020-02-15  2020-02-16  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-17  2020-02-18  2020-02-19  2020-02-20  2020-02-21  2020-02-22  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-23  2020-02-24  2020-02-25  2020-02-26  2020-02-27  2020-02-28  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-29  2020-03-01  2020-03-02  2020-03-03  2020-03-04  2020-03-05  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-03-06  2020-03-07  2020-03-08  2020-03-09  2020-03-10  2020-03-11  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0       

In [86]:
ccc_df = ccc_df[ccc_df['State'] != 'AK']
ccc_df = ccc_df[ccc_df['State'] != 'HI']
ccc_df = ccc_df[ccc_df['State'] != 'US']
ccc_df = ccc_df[ccc_df['State'] != 'PR']
ccc_df = ccc_df[ccc_df['countyFIPS'] != 0]

In [87]:
ccc_df.shape

(3108, 414)

In [88]:
# Check if all ccc_df fips in p_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

ccc_fips = []
for i, v in enumerate(ccc_df['countyFIPS']):
    ccc_fips.append(int(v))
    
for i, v in enumerate(ccc_fips):
    if v not in ideal_fips:
        print(v)

11001


In [89]:
# Check if all p_df fips in ccc_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

ccc_fips = []
for i, v in enumerate(ccc_df['countyFIPS']):
    ccc_fips.append(int(v))
    
for i, v in enumerate(ideal_fips):
    if v not in ccc_fips:
        print(v)

11


In [90]:
ccc_df.iloc[290, 0] = 11

In [91]:
ccc_df.shape

(3108, 414)

In [92]:
cd_df.head()

countyFIPS            County Name State  StateFIPS  2020-01-22  2020-01-23  \
0           0  Statewide Unallocated    AL          1           0           0   
1        1001        Autauga County     AL          1           0           0   
2        1003        Baldwin County     AL          1           0           0   
3        1005        Barbour County     AL          1           0           0   
4        1007           Bibb County     AL          1           0           0   

   2020-01-24  2020-01-25  2020-01-26  2020-01-27  2020-01-28  2020-01-29  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-01-30  2020-01-31  2020-02-01  2020-02-02  2020-02-03  2020-02-04  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-05  2020-02-06  2020-02-07  2020-02-08  2020-02-09  2020-02-10  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-11  2020-02-12  2020-02-13  2020-02-14  2020-02-15  2020-02-16  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-17  2020-02-18  2020-02-19  2020-02-20  2020-02-21  2020-02-22  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-23  2020-02-24  2020-02-25  2020-02-26  2020-02-27  2020-02-28  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-02-29  2020-03-01  2020-03-02  2020-03-03  2020-03-04  2020-03-05  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   2020-03-06  2020-03-07  2020-03-08  2020-03-09  2020-03-10  2020-03-11  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0       

In [93]:
cd_df = cd_df[cd_df['State'] != 'AK']
cd_df = cd_df[cd_df['State'] != 'HI']
cd_df = cd_df[cd_df['State'] != 'US']
cd_df = cd_df[cd_df['State'] != 'PR']
cd_df = cd_df[cd_df['countyFIPS'] != 0]

In [94]:
cd_df.shape

(3108, 414)

In [95]:
# Check if all cd_df fips in p_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

cd_fips = []
for i, v in enumerate(cd_df['countyFIPS']):
    cd_fips.append(int(v))
    
for i, v in enumerate(cd_fips):
    if v not in ideal_fips:
        print(v)

11001


In [96]:
# Check if all p_df fips in cd_df

ideal_fips = []
for i, v in enumerate(p_df['FIPS']):
    ideal_fips.append(v)

cd_fips = []
for i, v in enumerate(cd_df['countyFIPS']):
    cd_fips.append(int(v))
    
for i, v in enumerate(ideal_fips):
    if v not in cd_fips:
        print(v)

11


In [97]:
cd_df.iloc[290, 0] = 11

In [98]:
cd_df.shape

(3108, 414)

## Compile All Data

In [99]:
data = pd.DataFrame()
data['fips'] = p_df['FIPS']
data['state'] = p_df['State']
data['county'] = p_df['County']

In [100]:
data.head()

fips    state   county
1  1001  Alabama  Autauga
2  1003  Alabama  Baldwin
3  1005  Alabama  Barbour
4  1007  Alabama     Bibb
5  1009  Alabama   Blount

In [101]:
data.shape

(3108, 3)

In [102]:
# Add win data

wins = []
for i, v in enumerate(data['fips']):
    if v == 11:
        wins.append(county_wins[11001])
    elif v == 46102:
        wins.append(county_wins[46113])
    else:
        wins.append(county_wins[v])
data['wins'] = wins

In [103]:
data.shape

(3108, 4)

In [104]:
# Add the poverty rate data

categories = [['Poverty Rate ', 'poverty_rate_'],
              ['Population ', 'population_'],
              ['Persons in Poverty ', 'persons_in_poverty_']]

for i, v in enumerate(categories):
    for x in range(1960, 2020, 10):
        old_name = v[0] + str(x)
        new_name = v[1] + str(x)
        data[new_name] = ''
        for a, b in enumerate(data['fips']):
            for y, z in enumerate(p_df['FIPS']):
                if b == z:
                    data.iloc[a, -1] = list(p_df[old_name])[y]
                    break

In [105]:
# Add the unemployment rates and median household income for 2019

data['rural_urban_continuum_code_2013'] = list(u_df['Rural_urban_continuum_code_2013'])
data['urban_influence_code_2013'] = list(u_df['Urban_influence_code_2013'])
data['metro_2013'] = list(u_df['Metro_2013'])

categories = [['Civilian_labor_force_', 'civilian_labor_force_'],
              ['Employed_', 'employed_'],
              ['Unemployed_', 'unemployed_'],
              ['Unemployment_rate_', 'unemployment_rate_']]

for i, v in enumerate(categories):
    for x in range(2000, 2020, 1):
        old_name = v[0] + str(x)
        new_name = v[1] + str(x)
        data[new_name] = ''
        for a, b in enumerate(data['fips']):
            for y, z in enumerate(u_df['fips_txt']):
                if int(b) == int(z):
                    data.iloc[a, -1] = list(u_df[old_name])[y]
                    break
                    
data['median_household_income_2019'] = list(u_df['Median_Household_Income_2019'])
data['med_hh_income_percent_state_total_2019'] = list(u_df['Med_HH_Income_Percent_of_State_Total_2019'])

In [106]:
# Add the education rates

data['rural-urban_continuum_code_2003'] = e_df['2003 Rural-urban Continuum Code']
data['urban_influence_code_2003'] = e_df['2003 Urban Influence Code']
# not adding 2013 codes because already given in u_df

categories_1970 = [['no_diploma_', 'Less than a high school diploma, '],
              ['diploma_only_', 'High school diploma only, '],
              ['some_college_', 'Some college (1-3 years), '],
              ['completed_college_', 'Four years of college or higher, '],
              ['percent_no_diploma_', 'Percent of adults with less than a high school diploma, '],
              ['percent_diploma_only_', 'Percent of adults with a high school diploma only, '],
              ['percent_some_college_', 'Percent of adults completing some college (1-3 years), '],
              ['percent_completed_college_', 'Percent of adults completing four years of college or higher, ']]

categories_1990 = [['no_diploma_', 'Less than a high school diploma, '],
              ['diploma_only_', 'High school diploma only, '],
              ['some_college/associate_degree_', "Some college or associate's degree, "],
              ['bachelor_degree_or_higher_', "Bachelor's degree or higher, "],
              ['percent_no_diploma_', 'Percent of adults with less than a high school diploma, '],
              ['percent_diploma_only_', 'Percent of adults with a high school diploma only, '],
              ['percent_some_college/associate_degree_', "Percent of adults completing some college or associate's degree, "],
              ['percent_bachelor_degree_or_higher_', "Percent of adults with a bachelor's degree or higher, "]]

for i in range(8):
    for x in range(1970, 2020, 10):
        if x < 1990:
            old_name = categories_1970[i][1] + str(x)
            new_name = categories_1970[i][0] + str(x)
            data[new_name] = ''
            for a, b in enumerate(data['fips']):
                for y, z in enumerate(e_df['FIPS Code']):
                    if b == z:
                        data.iloc[a, -1] = list(e_df[old_name])[y]
                        break
        if x >= 1990:
            if x != 2010:
                old_name = categories_1990[i][1] + str(x)
                new_name = categories_1990[i][0] + str(x)
                data[new_name] = ''
                for a, b in enumerate(data['fips']):
                    for y, z in enumerate(e_df['FIPS Code']):
                        if b == z:
                            data.iloc[a, -1] = list(e_df[old_name])[y]
                            break
            elif x == 2010:
                old_name = categories_1990[i][1] + '2014-18'
                new_name = categories_1990[i][0] + '2014-2018'
                data[new_name] = ''
                for a, b in enumerate(data['fips']):
                    for y, z in enumerate(e_df['FIPS Code']):
                        if b == z:
                            data.iloc[a, -1] = list(e_df[old_name])[y]
                            break

In [107]:
# Add fips to d_df1

d_df1['fips'] = ''
for i, v in enumerate(d_df1['STATE']):
    d_df1.iloc[i, -1] = v*1000 + list(d_df1['COUNTY'])[i]

In [108]:
d_df1.head()

SUMLEV  STATE  COUNTY   STNAME         CTYNAME  YEAR  AGEGRP  TOT_POP  \
0        50      1       1  Alabama  Autauga County     1       0    54571   
228      50      1       3  Alabama  Baldwin County     1       0   182265   
456      50      1       5  Alabama  Barbour County     1       0    27457   
684      50      1       7  Alabama     Bibb County     1       0    22915   
912      50      1       9  Alabama   Blount County     1       0    57322   

     TOT_MALE  TOT_FEMALE  WA_MALE  WA_FEMALE  BA_MALE  BA_FEMALE  IA_MALE  \
0       26569       28002    21295      22002     4559       5130      119   
228     89196       93069    78334      81376     8396       8878      662   
456     14576       12881     7463       6524     6808       6133       93   
684     12301       10614     9055       8494     3079       1981       35   
912     28362       28960    27398      27970      420        397      178   

     IA_FEMALE  AA_MALE  AA_FEMALE  NA_MALE  NA_FEMALE  TOM_MALE  TOM_FEMALE  \
0          139      200        284       29         18       367         429   
228        677      540        829       72         49      1192        1260   
456         73       59         53       29         18       124          80   
684         33       13          9       28          5        91          92   
912        173       45         80       38         25       283         315   

     WAC_MALE  WAC_FEMALE  BAC_MALE  BAC_FEMALE  IAC_MALE  IAC_FEMALE  \
0       21633       22391      4704        5306       277         314   
228     79452       82576      8804        9327      1295        1346   
456      7546        6583      6898        6182       162         116   
684      9133        8567      3119        2028        84          76   
912     27670       28275       493         472       364         376   

     AAC_MALE  AAC_FEMALE  NAC_MALE  NAC_FEMALE  NH_MALE  NH_FEMALE  \
0         300         409        42          37    25875      27386   
228       783        1072       127          84    84771      89502   
456        67          57        33          23    13799      12271   
684        26          30        33           9    12003      10506   
912        81         134        45          31    25793      26903   

     NHWA_MALE  NHWA_FEMALE  NHBA_MALE  NHBA_FEMALE  NHIA_MALE  NHIA_FEMALE  \
0        20709        21485       4512         5091        103          115   
228      74314        78160       8268         8754        558          592   
456       6832         6017       6746         6084         31           31   
684       8798         8410       3065         1967         35           32   
912      24982        26009        368          363        134          151   

     NHAA_MALE  NHAA_FEMALE  NHNA_MALE  NHNA_FEMALE  NHTOM_MALE  NHTOM_FEMALE  \
0          194          280         13            9         344           406   
228        532          817         47           39        1052          1140   
456         59           53         16            8         115            78   
684         13            9          6            1          86            87   
912         44           76         11            7         254           297   

     NHWAC_MALE  NHWAC_FEMALE  NHBAC_MALE  NHBAC_FEMALE  NHIAC_MALE  \
0         21026         21853        4647          5258         251   
228       75299         79251        8648          9173        1106   
456        6909          6074        6832          6133          93   
684        8876          8480        3100          2010          83   
912       25228         26297         433           434         298   

     NHIAC_FEMALE  NHAAC_MALE  NHAAC_FEMALE  NHNAC_MALE  NHNAC_FEMALE  H_MALE  \
0             282         291           398          23            27     694   
228          1183         729          1024          82            58    4425   
456            72          66            57          19            13     777   
684    

In [109]:
# Add demographics data

# 2010 data
    
white_m = []
white_f = []
black_m = []
black_f = []
native_american_m = []
native_american_f = []
asian_m = []
asian_f = []
hispanic_m = []
hispanic_f = []

for i in range(3108):
        
    total_pop = d_df1.iloc[i, 7]

    white_male_pop = d_df1.iloc[i, 10]
    white_female_pop = d_df1.iloc[i, 11]
    white_m.append((d_df1.iloc[i, -1], (white_male_pop/total_pop)*100))
    white_f.append((d_df1.iloc[i, -1], (white_female_pop/total_pop)*100))

    black_male_pop = d_df1.iloc[i, 12]
    black_female_pop = d_df1.iloc[i, 13]
    black_m.append((d_df1.iloc[i, -1], (black_male_pop/total_pop)*100))
    black_f.append((d_df1.iloc[i, -1], (black_female_pop/total_pop)*100))

    native_american_male_pop = d_df1.iloc[i, 14]
    native_american_female_pop = d_df1.iloc[i, 15]
    native_american_m.append((d_df1.iloc[i, -1], (native_american_male_pop/total_pop)*100))
    native_american_f.append((d_df1.iloc[i, -1], (native_american_female_pop/total_pop)*100))

    asian_male_pop = d_df1.iloc[i, 16]
    asian_female_pop = d_df1.iloc[i, 17]
    asian_m.append((d_df1.iloc[i, -1], (asian_male_pop/total_pop)*100))
    asian_f.append((d_df1.iloc[i, -1], (asian_female_pop/total_pop)*100))

    hispanic_male_pop = d_df1.iloc[i, 56]
    hispanic_female_pop = d_df1.iloc[i, 57]
    hispanic_m.append((d_df1.iloc[i, -1], (hispanic_male_pop/total_pop)*100))
    hispanic_f.append((d_df1.iloc[i, -1], (hispanic_female_pop/total_pop)*100))

In [110]:
categories = [white_m, white_f, black_m, black_f, native_american_m, 
              native_american_f, asian_m, asian_f, hispanic_m, hispanic_f]

names = {'percent_white_male_2010': white_m, 'percent_white_female_2010': white_f, 'percent_black_male_2010': black_m, 
         'percent_black_female_2010': black_f, 'percent_native_american_male_2010': native_american_m, 
         'percent_native_american_female_2010': native_american_f, 'percent_asian_male_2010': asian_m, 
         'percent_asian_female_2010': asian_f, 'percent_hispanic_male_2010': hispanic_m, 
         'percent_hispanic_female_2010': hispanic_f}

for x in categories:
    new_name = ''
    for c in names.keys():
        if names[c] == x:
            new_name = c
            break
    data[new_name] = ''
    for a, b in enumerate(data['fips']):
        for y, z in enumerate(x):
            if z[0] == b:
                data.iloc[a, -1] = z[1]
                break
            elif z[0] == 11001:
                data.iloc[a, -1] = z[1]
                break

In [111]:
# Add fips to d_df2

d_df2['fips'] = ''
for i, v in enumerate(d_df2['STATE']):
    d_df2.iloc[i, -1] = v*1000 + list(d_df2['COUNTY'])[i]

In [112]:
d_df2.head()

SUMLEV  STATE  COUNTY   STNAME         CTYNAME  YEAR  AGEGRP  TOT_POP  \
209       50      1       1  Alabama  Autauga County    12       0    55869   
437       50      1       3  Alabama  Baldwin County    12       0   223234   
665       50      1       5  Alabama  Barbour County    12       0    24686   
893       50      1       7  Alabama     Bibb County    12       0    22394   
1121      50      1       9  Alabama   Blount County    12       0    57826   

      TOT_MALE  TOT_FEMALE  WA_MALE  WA_FEMALE  BA_MALE  BA_FEMALE  IA_MALE  \
209      27092       28777    20878      21729     5237       6000      121   
437     108247      114987    94810     100388     9486      10107      903   
665      13064       11622     6389       5745     6311       5595      103   
893      11929       10465     8766       8425     2941       1822       53   
1121     28472       29354    27258      28154      516        462      192   

      IA_FEMALE  AA_MALE  AA_FEMALE  NA_MALE  NA_FEMALE  TOM_MALE  TOM_FEMALE  \
209         145      286        370       32         26       538         507   
437         839      932       1448       74         80      2042        2125   
665          67       55         61       34         18       172         136   
893          50       23         25       22          4       124         139   
1121        178       85        100       42         25       379         435   

      WAC_MALE  WAC_FEMALE  BAC_MALE  BAC_FEMALE  IAC_MALE  IAC_FEMALE  \
209      21372       22191      5514        6270       274         301   
437      96730      102373     10346       11008      1818        1785   
665       6517        5863      6432        5685       182         115   
893       8869        8551      3010        1897       106         101   
1121     27608       28559       658         618       400         420   

      AAC_MALE  AAC_FEMALE  NAC_MALE  NAC_FEMALE  NH_MALE  NH_FEMALE  \
209        441         509        55          49    26208      27990   
437       1405        1946       180         187   102702     109998   
665         72          80        38          23    12435      11134   
893         49          48        28          12    11586      10185   
1121       147         165        60          37    25522      26722   

      NHWA_MALE  NHWA_FEMALE  NHBA_MALE  NHBA_FEMALE  NHIA_MALE  NHIA_FEMALE  \
209       20138        21077       5171         5927        105          138   
437       89845        95902       9308         9907        753          754   
665        5894         5341       6260         5547         52           43   
893        8482         8181       2912         1807         50           41   
1121      24494        25682        453          419        143          139   

      NHAA_MALE  NHAA_FEMALE  NHNA_MALE  NHNA_FEMALE  NHTOM_MALE  \
209         282          364         20           20         492   
437         911         1435         53           70        1832   
665          55           61         21           10         153   
893          21           25          5            1         116   
1121         73           90         14            7         345   

      NHTOM_FEMALE  NHWAC_MALE  NHWAC_FEMALE  NHBAC_MALE  NHBAC_FEMALE  \
209            464       20594         21504        5425          6177   
437           1930       91586         97727       10078         10727   
665            132        6008          5455        6369          5635   
893            130        8578          8298        2978          1878   
1121           385       24814         26043         582           560   

      NHIAC_MALE  NHIAC_FEMALE  NHAAC_MALE  NHAAC_FEMALE  NHNAC_MALE  \
209          234           274         426           490          39   
437         1554          1588        1336          1881         125   
665          121            89          71            80          24   
893          100            86          44            47          11

In [113]:
# 2019 data

white_m = []
white_f = []
black_m = []
black_f = []
native_american_m = []
native_american_f = []
asian_m = []
asian_f = []
hispanic_m = []
hispanic_f = []

for i in range(3108):
        
    total_pop = d_df2.iloc[i, 7]

    white_male_pop = d_df2.iloc[i, 10]
    white_female_pop = d_df2.iloc[i, 11]
    white_m.append((d_df2.iloc[i, -1], (white_male_pop/total_pop)*100))
    white_f.append((d_df2.iloc[i, -1], (white_female_pop/total_pop)*100))

    black_male_pop = d_df2.iloc[i, 12]
    black_female_pop = d_df2.iloc[i, 13]
    black_m.append((d_df2.iloc[i, -1], (black_male_pop/total_pop)*100))
    black_f.append((d_df2.iloc[i, -1], (black_female_pop/total_pop)*100))

    native_american_male_pop = d_df2.iloc[i, 14]
    native_american_female_pop = d_df2.iloc[i, 15]
    native_american_m.append((d_df2.iloc[i, -1], (native_american_male_pop/total_pop)*100))
    native_american_f.append((d_df2.iloc[i, -1], (native_american_female_pop/total_pop)*100))

    asian_male_pop = d_df2.iloc[i, 16]
    asian_female_pop = d_df2.iloc[i, 17]
    asian_m.append((d_df2.iloc[i, -1], (asian_male_pop/total_pop)*100))
    asian_f.append((d_df2.iloc[i, -1], (asian_female_pop/total_pop)*100))

    hispanic_male_pop = d_df2.iloc[i, 56]
    hispanic_female_pop = d_df2.iloc[i, 57]
    hispanic_m.append((d_df2.iloc[i, -1], (hispanic_male_pop/total_pop)*100))
    hispanic_f.append((d_df2.iloc[i, -1], (hispanic_female_pop/total_pop)*100))

In [114]:
categories = [white_m, white_f, black_m, black_f, native_american_m, 
              native_american_f, asian_m, asian_f, hispanic_m, hispanic_f]

names = {'percent_white_male_2019': white_m, 'percent_white_female_2019': white_f, 'percent_black_male_2019': black_m, 
         'percent_black_female_2019': black_f, 'percent_native_american_male_2019': native_american_m, 
         'percent_native_american_female_2019': native_american_f, 'percent_asian_male_2019': asian_m, 
         'percent_asian_female_2019': asian_f, 'percent_hispanic_male_2019': hispanic_m, 
         'percent_hispanic_female_2019': hispanic_f}

for x in categories:
    new_name = ''
    for c in names.keys():
        if names[c] == x:
            new_name = c
            break
    data[new_name] = ''
    for a, b in enumerate(data['fips']):
        for y, z in enumerate(x):
            if z[0] == b:
                data.iloc[a, -1] = z[1]
                break
            elif z[0] == 11001:
                data.iloc[a, -1] = z[1]
                break

In [115]:
# Add COVID-19 data

data['cumulative_confirmed_cases'] = ''
for a, b in enumerate(data['fips']):
    for y, z in enumerate(ccc_df['countyFIPS']):
        if b == z:
            data.iloc[a, -1] = ccc_df.iloc[y, 4:414].sum()
    
data['cumulative_deaths'] = ''
for a, b in enumerate(data['fips']):
    for y, z in enumerate(ccc_df['countyFIPS']):
        if b == z:
            data.iloc[a, -1] = cd_df.iloc[y, 4:414].sum()

In [116]:
data.isna().sum()

fips                                                0
state                                               0
county                                              0
wins                                                0
poverty_rate_1960                                   0
poverty_rate_1970                                   0
poverty_rate_1980                                   0
poverty_rate_1990                                   0
poverty_rate_2000                                   0
poverty_rate_2010                                   0
population_1960                                    15
population_1970                                     7
population_1980                                     4
population_1990                                     2
population_2000                                     1
population_2010                                     0
persons_in_poverty_1960                            15
persons_in_poverty_1970                             7
persons_in_poverty_1980     

In [117]:
data[data['wins'] == 6]

fips          state        county  wins poverty_rate_1960  \
324   10001       Delaware          Kent     6              24.3   
1350  27027      Minnesota          Clay     6              15.6   
1626  30005        Montana        Blaine     6              34.7   
1798  33011  New Hampshire  Hillsborough     6              12.8   
1875  36031       New York         Essex     6              25.8   
1904  36091       New York      Saratoga     6              17.2   
3005  53009     Washington       Clallam     6              16.8   
3111  55029      Wisconsin          Door     6              21.9   

     poverty_rate_1970 poverty_rate_1980 poverty_rate_1990 poverty_rate_2000  \
324               15.4              13.6              11.3              10.7   
1350              11.6                11              15.9              13.2   
1626              31.4              24.1              27.7              28.1   
1798               8.2               7.2               5.9               6.3   
1875                14              13.4              12.3              11.6   
1904               8.3                 8               5.9               5.7   
3005              11.3              10.2              12.5              12.5   
3111                12               6.9               9.8               6.4   

     poverty_rate_2010 population_1960 population_1970 population_1980  \
324               12.4           60412           76823           93956   
1350              12.5           37561           42982           44851   
1626              27.1            7887            6600            6878   
1798                 8          172763          219433          270936   
1875              12.4           34362           34353           35454   
1904               6.5           87685          119161          150699   
3005              13.5           29001           34192           50743   
3111              11.4           20944           19876           24612   

     population_1990 population_2000 population_2010 persons_in_poverty_1960  \
324           106673          122717          157929                   14668   
1350           46282           47338           54900                    5858   
1626            6596            6945            6425                    2733   
1798          328996          372902          393052                   22157   
1875           34767           35998           36728                    8865   
1904          176816          196387          215731                   15093   
3005           54705           62602           69886                    4875   
3111           25318           27555           27338                    4585   

     persons_in_poverty_1970 persons_in_poverty_1980 persons_in_poverty_1990  \
324                    11813                   12797                   12071   
1350                    4966                    4941                    7355   
1626                    2073                    1659                    1825   
1798                   18014                   19394                   19261   
1875                    4802                    4760                    4263   
1904                    9916                   12058                   10509   
3005                    3856                    5172                    6852   
3111                    2391                    1703                    2487   

     persons_in_poverty_2000 persons_in_poverty_2010  \
324                    13083                   19550   
1350                    6272                    6851   
1626                    1951                    1740   
1798                   23358                   31481   
1875                    4173                    4548   
1904                   11238                   13922   
3005                    7825                    9401   
3111                    1763                    3120   

     rural_urban_continuum_code_2013 urban_influence_code_2

In [118]:
data.head()

fips    state   county  wins poverty_rate_1960 poverty_rate_1970  \
1  1001  Alabama  Autauga     3              53.1              23.7   
2  1003  Alabama  Baldwin     3              41.8              22.6   
3  1005  Alabama  Barbour     3              65.7              43.7   
4  1007  Alabama     Bibb     3              56.8              38.2   
5  1009  Alabama   Blount     3              45.6              24.2   

  poverty_rate_1980 poverty_rate_1990 poverty_rate_2000 poverty_rate_2010  \
1              16.6              15.7              10.9              11.6   
2              16.2              14.3              10.1              13.3   
3              30.8              25.2              26.8              26.1   
4              21.2              21.2              20.6              16.5   
5                18              15.3              11.7              14.7   

  population_1960 population_1970 population_1980 population_1990  \
1           18454           24313           32107           34009   
2           49722           58776           77726           97227   
3           24975           22424           24533           24785   
4           14775           13761           15520           16304   
5           25414           26746           36244           38924   

  population_2000 population_2010 persons_in_poverty_1960  \
1           43377           54255                    9807   
2          138148          180970                   20773   
3           26239           24530                   16395   
4           19852           21794                    8386   
5           50499           56993                   11595   

  persons_in_poverty_1970 persons_in_poverty_1980 persons_in_poverty_1990  \
1                    5758                    5337                    5324   
2                   13304                   12616                   13947   
3                    9788                    7559                    6244   
4                    5256                    3296                    3454   
5                    6462                    6527                    5938   

  persons_in_poverty_2000 persons_in_poverty_2010  \
1                    4738                    6267   
2                   14018                   24002   
3                    7032                    6395   
4                    4091                    3594   
5                    5930                    8396   

  rural_urban_continuum_code_2013 urban_influence_code_2013 metro_2013  \
1                               2                         2          1   
2                               3                         2          1   
3                               6                         6          0   
4                               1                         1          1   
5                               1                         1          1   

  civilian_labor_force_2000 civilian_labor_force_2001  \
1                     21720                     21955   
2                     69533                     69161   
3                     11373                     11250   
4                      8565                      9081   
5                     25106                     25305   

  civilian_labor_force_2002 civilian_labor_force_2003  \
1                     22094                     22604   
2                     69169                     72299   
3                     10971                     10977   
4                      8933                      8836   
5                     25757                     25900   

  civilian_labor_force_2004 civilian_labor_force_2005  \
1                     23218                     23949   
2                     74772                     76804   
3                     10633                     10760   
4                      8843                      8861   
5                     26208                     26446   

  civilian_labor_force_2006 civilian_labor_force_2007  \
1                 

In [119]:
data.shape

(3108, 171)

In [120]:
data.to_csv('data/compiled_data/main_data.csv') # main data